In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import mouse_utils as mu
import torch

In [3]:
df = pd.read_csv("./data/K-Data.csv")


v1 = df.query("region == 'V1'")
m = v1.m.unique()[2]
v1_data = v1[v1.m == m]
# v1_data = v1_data.query("grat_spat_freq == 0.332966").query("grat_phase == [180]")
v1_data = v1_data[['u', 'unit_type', 'grat_orientation', 'grat_contrast', 'response', 'smoothed_response']].reset_index(drop=True)
v1_data = v1_data.groupby(['unit_type','u', 'grat_contrast', 'grat_orientation'], as_index=False).mean()


# Get unique values for each column
unique_u = v1_data['u'].unique()
unique_contrast = v1_data['grat_contrast'].unique()
unique_orientation = v1_data['grat_orientation'].unique()


# Create a 3D numpy array filled with NaN values
shape = (len(unique_u), len(unique_contrast), len(unique_orientation))
result_array = np.full(shape, np.nan)


# Iterate through the DataFrame and fill the array
for index, row in v1_data.iterrows():
    u_index = np.where(unique_u == row['u'])[0][0]
    contrast_index = np.where(unique_contrast == row['grat_contrast'])[0][0]
    orientation_index = np.where(unique_orientation == row['grat_orientation'])[0][0]
    
    result_array[u_index, contrast_index, orientation_index] = row['response']


/var/folders/78/9j91h4q516z7774553xzp19m0000gn/T/ipykernel_76775/3572531747.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/K-Data.csv")


In [4]:

# J_array = [1.99, 1.9, 1.01, 0.79]  # Need to make parameters of an exponential
# P_array = [0.11, 0.11, 0.45, 0.45]
# w_array = [32., 32., 32., 32.]


# J_array = [0.69, 0.64, 0., -0.29] # Max log values
# P_array = [-2.21, -2.21, -0.8, -0.8]
# w_array = [3.46, 3.46, 3.46, 3.46]

J_array = [0, -0.29, 0.69, -0.64] # Keen log values
P_array = [-0.8, -2.21, -2.21, -0.8]
w_array = [3.46, 3.46, 3.46, 3.46]

# J_array = [-5.1707, -0.0277,  0.9482,  0.1585]
# P_array = [-6.1361, -1.9772,  0.8548,  1.2384]
# w_array = [0.3990, 3.6245, 4.1893, 3.6955]

# J_array = [0.69, 0.64, 0., -0.29]
# P_array = [-1.21, -1.21, -0.8, -0.8]
# w_array = [3.46, 3.46, 3.46, 3.46]

# J_array = [9.04e-02, 3.82e-05, 7.62e-5, 2.52]
# P_array = [1.93e-02,  8.78,  1.79e-04,  2.85]
# w_array = [8.78, 166, 1.23e-2, 1.81e2]


J_array = [-3.5, 0.346, 0.381, -0.669]  # JAX minima 
P_array = [-0.5549,  -2.64767,  -2.7836,  -1.597]
w_array = [4.26, 3.44, 3.30, 3.47]




model = mu.NeuroNN(J_array, P_array, w_array, 7000)
if torch.cuda.is_available():
    model = model.to('cuda')
    print("Model moved to GPU.")
else:
    print("GPU not available. Keeping the model on CPU.")

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

mu.training_loop(model, optimizer, result_array)

GPU not available. Keeping the model on CPU.
